# Figure panel 4

In [1]:
options(warn=-1)

In [2]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # TradeSeq
            library(CellChat),

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(ComplexHeatmap), 
            library(circlize), 
            library(viridis), 
            library(ggplotify), 
            library(ggrepel), 
            library(cowplot), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [3]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [4]:
random_seed <- 42
set.seed(random_seed)

In [5]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [6]:
# Source
source("bin/so_pl.R")
source("bin/dea_pl.R")
source("bin/gsea_pp.R")
source("bin/gsea_pl.R")
source("bin/cci_pp.R")
source("bin/cci_pl.R")

In [7]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Load reference data 

In [8]:
tf <- read.table("/research/peer/fdeckert/reference/animaltfdb/Mus_musculus_TF.txt")[[2]]

lr <- CellChat::CellChatDB.mouse[[1]] %>% dplyr::select(pathway_name, ligand, receptor.symbol, receptor.family, annotation) %>% dplyr::filter(annotation %in% c("Cell-Cell Contact", "Secreted Signaling", "ECM-Receptor")) %>% 
    separate_rows(receptor.symbol, sep=", ") %>% dplyr::distinct() %>%
    separate_rows(ligand, sep=", ") %>% dplyr::distinct() %>% 
    dplyr::rename(pathway=pathway_name, receptor=receptor.symbol, family=receptor.family)

# DEA overview 

In [25]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.25

In [26]:
contrasts_vec <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [27]:
contrasts_vec <- c("IFNAR_fl_CpG_D3", "IFNAR_fl_Baseline_D0")
dea_res_2 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [28]:
mat_1 <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(p_val_adj <= adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t()
mat_2 <- lapply(dea_res_2, function(x) {x %>% dplyr::filter(p_val_adj <= adj_pval_thr & abs(avg_log2FC)>=log2FC_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t()

In [29]:
mat <- cbind(mat_1, mat_2) %>% data.frame() %>% dplyr::rename(Ctr_vs_D1=X1, Ctr_vs_D3=X2)

In [30]:
pdf("result/figures/1_scRNAseq/panel_4/hm_dea_cg.pdf", width=1.5, height=3)

draw(dea_count_hm(as.matrix(mat), fontsize_select=2))

dev.off()

pdf 
  2

# DEA Volcano plots  

## D1 vs Ctl

In [147]:
contrasts_vec <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [148]:
labels <- list(
    
)

In [149]:
vp_1 <- lapply(names(dea_res_1), function(i) {

    if(i %in% names(labels)) {

        label <- labels[[i]]
        
    } else {

        label <- NULL
        
    }

    v_pl(dea_res_1[[i]], title=i, log2FC_thr=log2FC_thr , adj_pval_thr=adj_pval_thr, color_pos=color$sample_group[contrasts_vec[1]], color_neg=color$sample_group[contrasts_vec[2]], label=label, top_label=15, label_merge=FALSE, point_size=0.5, label_size=2, aspect_ratio=0.8, y_limit=5) + theme(legend.position="none") + theme_global_set(size_select=4)

}
              )

In [150]:
pdf("result/figures/1_scRNAseq/panel_4/v_pl_d1_vs_ctl.pdf", width=5*2, height=4*2)

plots <- lapply(vp_1, function(p) egg::set_panel_size(p, width=unit(4, "cm"), height=unit(0.8*4, "cm")))
do.call(gridExtra::grid.arrange, c(plots, ncol=5, nrow=4, newpage=FALSE))

dev.off()

pdf 
  2

## D3 vs Ctl 

In [143]:
contrasts_vec <- c("IFNAR_fl_CpG_D3", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [144]:
labels <- list(
    
)

In [145]:
vp_1 <- lapply(names(dea_res_1), function(i) {

    if(i %in% names(labels)) {

        label <- labels[[i]]
        
    } else {

        label <- NULL
        
    }

    v_pl(dea_res_1[[i]], title=i, log2FC_thr=log2FC_thr , adj_pval_thr=adj_pval_thr, color_pos=color$sample_group[contrasts_vec[1]], color_neg=color$sample_group[contrasts_vec[2]], label=label, top_label=15, label_merge=FALSE, point_size=0.5, label_size=2, aspect_ratio=0.8, y_limit=5) + theme(legend.position="none") + theme_global_set(size_select=4)

}
              )

In [146]:
pdf("result/figures/1_scRNAseq/panel_4/v_pl_d3_vs_ctl.pdf", width=5*2, height=4*2)

plots <- lapply(vp_1, function(p) egg::set_panel_size(p, width=unit(4, "cm"), height=unit(0.8*4, "cm")))
do.call(gridExtra::grid.arrange, c(plots, ncol=5, nrow=4, newpage=FALSE))

dev.off()

pdf 
  2

# CellChat overview 

In [131]:
cco_1 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
cco_2 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")

In [65]:
cco_diff_1 <- list(IFNAR_fl_Baseline_D0=cco_1, IFNAR_fl_CpG_D1=cco_2)
cco_diff_1 <- mergeCellChat(cco_diff_1, add.names=names(cco_diff_1))
cco_diff_1@meta$sample_group <- factor(cco_diff_1@meta$sample_group, levels=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D1"))

Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



In [66]:
cco_diff_2 <- list(IFNAR_fl_Baseline_D0=cco_1, IFNAR_fl_CpG_D3=cco_2)
cco_diff_2 <- mergeCellChat(cco_diff_2, add.names=names(cco_diff_2))
cco_diff_2@meta$sample_group <- factor(cco_diff_2@meta$sample_group, levels=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_CpG_D3"))

Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



## Interaction count 

In [67]:
source <- NULL
target <- c("MEP", "Proerythroblast", "Erythroblast")

In [68]:
pdf("result/figures/1_scRNAseq/panel_4/hm_net_ctr_cg.pdf", width=2.5, height=2.5)

gridExtra::grid.arrange(
    
   hm_net(cellchat=cco_1, slot="count", source=source, target=target, title="Interaction count Ctr", breaks_max=30, fontsize_select=2) %>% ggplotify::as.ggplot()

)

dev.off()

pdf 
  2

In [69]:
pdf("result/figures/1_scRNAseq/panel_4/hm_net_d1_cg.pdf", width=2.5, height=2.5)

gridExtra::grid.arrange(
    
   hm_net(cellchat=cco_2, slot="count", source=source, target=target, title="Interaction count D1", breaks_max=30, fontsize_select=2) %>% ggplotify::as.ggplot()

)

dev.off()

pdf 
  2

In [70]:
pdf("result/figures/1_scRNAseq/panel_4/hm_net_d3_cg.pdf", width=2.5, height=2.5)

gridExtra::grid.arrange(
    
   hm_net(cellchat=cco_3, slot="count", source=source, target=target, title="Interaction count D3", breaks_max=30, fontsize_select=2) %>% ggplotify::as.ggplot()

)

dev.off()

pdf 
  2

# CCI DEA 

In [71]:
# Parameter 
pval_prob_thr <- 0.05
p_val_adj_dea <- 0.05
pathway_name_thr <- c("ICAM", "VCAM", "SIRP", "KIT", "TGFb", "TNF", "CXCL", "CCL")

# CellChat results
lr_res <- read.csv("result/cci/lr_res.csv", row.names=1)

# Add reversed Cell-cell interactions 
db <- CellChat::CellChatDB.mouse

# Impute custom interactions
options(stringsAsFactors=FALSE)
interaction <- read.csv("data/reference/cellchat/interaction.csv", row.names=1, na.strings=c(""))
interaction[is.na(interaction)] <- ""
db$interaction <- rbind(db$interaction, interaction)

# Get cell-cell contact interaction 
db <- CellChat::subsetDB(db, search=c("Cell-Cell Contact"), key="annotation")

# Subset lr results for cell-cell contact forward 
lr_res_ccc_fwd <- lr_res %>% dplyr::filter(lr_res$interaction_name %in% db$interaction$interaction_name)

# Reverse cell-cell contact 
lr_res_ccc_rev <- data.frame(

    source=lr_res_ccc_fwd$target, 
    target=lr_res_ccc_fwd$source, 
    interaction_name=paste0(lr_res_ccc_fwd$receptor_symbol, "_", lr_res_ccc_fwd$ligand_symbol), 
    prob=lr_res_ccc_fwd$prob, 
    pval=lr_res_ccc_fwd$pval, 
    pathway_name=lr_res_ccc_fwd$pathway_name, 
    ligand_symbol=lr_res_ccc_fwd$receptor_symbol, 
    receptor_symbol=lr_res_ccc_fwd$ligand_symbol, 
    sample_group=lr_res_ccc_fwd$sample_group
    
)

# Combine results 
lr_res_ccc <- rbind(lr_res_ccc_fwd, lr_res_ccc_rev)

# Assemble LR results 
lr_res <- rbind(lr_res[!lr_res$interaction_name %in% lr_res_ccc$interaction_name, ], lr_res_ccc)

# Subset by interaction pathway
lr_res <- lr_res[lr_res$pathway_name %in% pathway_name_thr, ]
lr_res$pathway_name <- factor(lr_res$pathway_name, levels=pathway_name_thr)

# DEA results 

In [72]:
dea_res_1 <- readRDS("result/dea/scRNAseq/sample_group/IFNAR_fl_CpG_D1_vs_IFNAR_fl_Baseline_D0.rds")[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_res_1 <- lapply(dea_res_1, function(x) {list("D1"=x)})

In [73]:
dea_res_2 <- readRDS("result/dea/scRNAseq/sample_group/IFNAR_fl_CpG_D1_vs_IFNAR_fl_Baseline_D0.rds")[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_res_2 <- lapply(dea_res_2, function(x) {list("D3"=x)})

In [74]:
dea_res <- Map(c, dea_res_1, dea_res_2)

## PTPG B (+/+) vs D1 (+/+) and D3 (+/+)

## cMo

In [75]:
source <- c("cMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [76]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [77]:
pdf("result/figures/1_scRNAseq/panel_4/cci_cmo_mep_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

In [78]:
source <- c("cMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [79]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [80]:
pdf("result/figures/1_scRNAseq/panel_4/cci_cmo_proeb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

In [81]:
source <- c("cMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [82]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [83]:
pdf("result/figures/1_scRNAseq/panel_4/cci_cmo_eb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

## intMo

In [84]:
source <- c("intMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [85]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [86]:
pdf("result/figures/1_scRNAseq/panel_4/cci_intmo_mep_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

In [87]:
source <- c("intMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [88]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [89]:
pdf("result/figures/1_scRNAseq/panel_4/cci_intmo_proeb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

In [90]:
source <- c("intMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [91]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [92]:
pdf("result/figures/1_scRNAseq/panel_4/cci_intmo_eb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

## RPM

In [93]:
source <- c("RPM")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [94]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [95]:
pdf("result/figures/1_scRNAseq/panel_4/cci_rpm_mep_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

In [96]:
source <- c("RPM")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [97]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [98]:
pdf("result/figures/1_scRNAseq/panel_4/cci_rpm_proeb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

In [99]:
source <- c("RPM")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [100]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [101]:
pdf("result/figures/1_scRNAseq/panel_4/cci_rpm_eb_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_res, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

pdf 
  2

# Interaction plots

In [102]:
sources <- c("RPM", "intMo", "cMo")
targets <- c("MEP", "Proerythroblast", "Erythroblast")

In [103]:
cco_1 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
cco_2 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")
cco_4 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_baseline_d0.rds")
cco_5 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d1.rds")
cco_6 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d3.rds")

In [104]:
cco_1 <- subsetCellChat(object=cco_1, idents.use=c(sources, targets))
cco_2 <- subsetCellChat(object=cco_2, idents.use=c(sources, targets))
cco_3 <- subsetCellChat(object=cco_3, idents.use=c(sources, targets))
cco_4 <- subsetCellChat(object=cco_4, idents.use=c(sources, targets))
cco_5 <- subsetCellChat(object=cco_5, idents.use=c(sources, targets))
cco_6 <- subsetCellChat(object=cco_6, idents.use=c(sources, targets))

The subset of cell groups used for CellChat analysis are  MEP Proerythroblast Erythroblast cMo intMo RPM 
Update slots object@images, object@net, object@netP in a single dataset... 
The subset of cell groups used for CellChat analysis are  MEP Proerythroblast Erythroblast cMo intMo RPM 
Update slots object@images, object@net, object@netP in a single dataset... 
The subset of cell groups used for CellChat analysis are  MEP Proerythroblast Erythroblast cMo intMo RPM 
Update slots object@images, object@net, object@netP in a single dataset... 
The subset of cell groups used for CellChat analysis are  MEP Proerythroblast Erythroblast cMo intMo RPM 
Update slots object@images, object@net, object@netP in a single dataset... 
The subset of cell groups used for CellChat analysis are  MEP Proerythroblast Erythroblast cMo intMo RPM 
Update slots object@images, object@net, object@netP in a single dataset... 
The subset of cell groups used for CellChat analysis are  MEP Proerythroblast Erythroblast

In [105]:
levels(cco_1@idents) <- c(targets, sources)
levels(cco_2@idents) <- c(targets, sources)
levels(cco_3@idents) <- c(targets, sources)
levels(cco_4@idents) <- c(targets, sources)
levels(cco_5@idents) <- c(targets, sources)
levels(cco_6@idents) <- c(targets, sources)

In [106]:
color_use <- color$celltype_low[names(color$celltype_low) %in% c(sources, targets)]

In [107]:
cco_list <- list(cco_1, cco_2, cco_3, cco_4, cco_5, cco_6)

q_prob_thr <- quantile(c(cco_1@net$prob, cco_2@net$prob, cco_3@net$prob), 0.90)
for (i in seq_along(cco_list)) {
    
    prob_arr <- cco_list[[i]]@net$prob
    prob_arr[prob_arr > q_prob_thr] <- q_prob_thr
    cco_list[[i]]@net$prob <- prob_arr
    
}

In [108]:
pdf("result/figures/1_scRNAseq/panel_4/cp_cci_net_ctl_cg.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[1]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

pdf 
  2

In [109]:
pdf("result/figures/1_scRNAseq/panel_4/cp_cci_net_d1_cg.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[2]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

pdf 
  2

In [110]:
pdf("result/figures/1_scRNAseq/panel_4/cp_cci_net_d3_cg.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[3]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

pdf 
  2